# Overall approach
This is a tool for classifying precontrast and 20s postcontrast MRIs (into LI-RADS stages / predicting survival?).

#### Training
-In mini-batches:
1. Import pre/20s DICOMs
2. Register pre to 20s
3. Segment whole liver
4. Obtain labels for HCC/non-HCC, as well as (HCC severity or survival?)
5. For HCC labels, calculate and store embeddings (feature values)
6. Train CNN on all

-After CNN is trained, then:
7. Train feature engineered network on HCC labels

#### Test pipeline:
1. Import pre/20s DICOMs
2. Register pre to 20s
3. Segment whole liver
4. Use CNN to classify as HCC/non-HCC
5. If HCC, calculate features values
6. If HCC, use feature engineered network to predict label

#### CNN models
First, try VGGNet, then ResNet. Then try VAE-GAN, training on benign only; deviation score = malignancy.

In [ ]:
import requests

In [5]:
import nibabel, argparse, matplotlib, sys
import numpy as np
import matplotlib.pyplot as plt
from skimage import measure
from mpl_toolkits.mplot3d import Axes3D
from pylab import get_cmap

In [ ]:
def draw_fig(image_array, name):
    """Draw an image of type np array and save it to."""

    image_array = image_array[:,:,slice_index]
    aspect = float(image_array.shape[1]) / image_array.shape[0]
    w = 20
    h = int(aspect * w)

    image_array = np.rot90(image_array)

    fig = plt.figure(frameon=False)
    fig.set_size_inches(w,h)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.imshow(image_array, interpolation='bilinear', norm=cnorm, cmap=plt.cm.gray, aspect='auto')

    img_fname = name + '.png'
    plt.savefig(img_fname)
    print('Segmentation saved as %s' % img_fname)

In [ ]:
def create_features(name, precontrast, arterial, liver_mask_file, tumor_mask_file, format_output, header_output, seg, slice_index):
    diff = art_data - pre_data # Calculate the subtracted image.
    diff[diff < 0] = 0.0  # The pre-contrast should never be more than the arterial. Clamp negative values to zero.

    x, y, z = diff.shape

    ##### Begin code for drawing specific mask slices. ######
    cnorm = matplotlib.colors.Normalize(vmin=0, vmax=np.amax(diff))

    draw_fig(diff, 'whole')

    # Calculate the liver volume.
    liver_volume = np.sum(liver) * pre_dimx * pre_dimy * pre_dimz
    is_in_mm = pre_units == 2 or liver_volume > 10000
    if is_in_mm: # 2 is the code for millimeter, we convert to cubic centimeters.
        liver_volume /= 1000

    tumor_volume = np.sum(tumor) * pre_dimx * pre_dimy * pre_dimz
    if is_in_mm: # 2 is the code for millimeter, we convert to cubic centimeters.
        tumor_volume /= 1000

    mean_intensity = np.mean(diff[img > 0])
    std_intensity = np.std(diff[img > 0])

    # Calculate the volume of tumor that whose voxel intensities are greater than the mean liver intensity.
    # Note this is NOT the same threshold as qEASL (which would need a parenchymal ROI in order to define the intensity threshold).
    enhancing_tumor_volume = np.sum(tumor[just_tumor > mean_liver_intensity]) * pre_dimx * pre_dimy * pre_dimz
    if is_in_mm: # 2 is the code for millimeter, we convert to cubic centimeters.
        enhancing_tumor_volume /= 1000

    headers = ['Name', 'Liver volume', 'Mean liver intensity', 'STD liver intensity', 'Tumor volume', 'Mean tumor intensity', 'STD tumor intensity', 'Enhancing tumor volume']
    features = [name, liver_volume, mean_liver_intensity, std_liver_intensity, tumor_volume, mean_tumor_intensity, std_tumor_intensity, enhancing_tumor_volume]
    if format_output == 'tabulate':
        if header_output:
            results = tabulate.tabulate([features], headers=headers)
        else:
            results = tabulate.tabulate([features])
    else:
        features_row = '\t'.join(str(feature) for feature in features)
        if header_output:
            results = '\t'.join(headers) + '\n' + features_row
        else:
            results = features_row

    print(results)

In [ ]:
def get_dcms(host='vnatest1vt', port='8082', user='user', pw='pass'):
    query_str = '&'.join(['requestType=WADO','contenttype=image/jpeg'])
    #'AcuoREST/search='
    #'0x00080060=ct' modality
    #'0x00100010=a*' names beginning with a
    #'&studyUID='
    #'&seriesUID='
    #'&objectUID='
    url = ''.join(['http://', host, ':', port, '/AcuoWadoHandler/wadoget?', query_str])
    
    r = requests.get(url, auth=(user, pw))
    
    return r.json()

In [ ]:
def get_report(user='user', pw='pass', mrns=None):
    query_str = '&'.join(['format=json', 'exam_type=mri'])
    
    if mrns is not None:
        query_str += '&patient_mrn__in='+','.join(mrns)
        
    url = 'https://demo.montagehealthcare.com/api/v1/report/?' + query_str
    
    r = requests.get(url, auth=(user, pw))
    
    return r.json()

In [ ]:
def ni_load(ni_fn):
    """
    Load a nifti image along with its dimensions.
    
    img is the normalized (0-255) image
    dims is the spacing between pixels
    dim_units is the unit of spacing (e.g. millimeters or centimeters)
    """
    
    img = nibabel.load(ni_fn)
    img = nibabel.as_closest_canonical(img) # make sure it is in the correct orientation
    
    dims = img.header['pixdim'][1:4]
    dim_units = img.header['xyzt_units']
    
    img = np.asarray(img.dataobj).astype(dtype='float64')
    img = img[::-1,:,:] # Orient the image along the same axis as the binary masks.
    img =  (255 * (img / np.amax(img))).astype(dtype='uint8')
    
    return img, dims, dim_units

def apply_mask(img, mask_file):
    """Apply the mask in mask_file to img and return the masked image."""
    with open(mask_file, 'rb') as f:
        mask = f.read()
        mask = np.fromstring(mask, dtype='uint8')
        mask = np.reshape(mask, img.shape, order='F')
        mask = mask[:,::-1,:]
    
    img[mask <= 0] = 0
    
    return img

In [ ]:
def training_pipeline(pre_dcm_fns, post_dcm_fns, labels):
    """
    pre_dcm_fn is the precontrast image, and post_dcm_fn is the postcontrast image
    1. Import pre/20s DICOMs
    2. Register pre to 20s
    3. Segment whole liver
    4. Obtain labels for HCC/non-HCC, as well as (HCC severity or survival?)
    5. For HCC labels, calculate and store embeddings (feature values)
    6. Train CNN on all
    7. Train feature engineered network on HCC labels
    """
    train_frac = 0.7
    data_size = len(pre_dcm_fns)
    
    # TODO: do a proper k-fold x-validation split
    X_cnn_train = X_cnn[:data_size*train_frac]
    X_cnn_val = X_cnn[data_size*train_frac:]
    
    for x in range(data_size):
        pre_dcm_fn = pre_dcm_fns[x]
        post_dcm_fn = post_dcm_fn[x]
        label = labels[x]
        
        pre_img = dcm_load(pre_dcm_fn)
        post_img = dcm_load(post_dcm_fn)

        pre_img = seg_liver(pre_img)
        post_img = seg_liver(post_img)
        
        y_cnn = label['is-hcc']
        label.pop('is-hcc')
        X_cnn = [pre_img, post_img, label]
        
        """if y_cnn:
            features = get_features(pre_img, post_img, label)
            
            # rfm is random forest model
            y_rfm = label
            X_rfm = [pre_img, post_img, label]"""

In [ ]:
def dcm_2_ni(fn):
    pass

In [ ]:
def reg_dcms(moving, target):
    pass

def seg_liver(img):
    pass